# Humid Heat Metrics

Wet Bulb Glove Temperature ($WBGT$) is a measure of heat stress outdoors ($WBGT_{OD}$) or indoors/in the shade ($WBGT_{ID}$).

Dry Bulb  Ambient Temperatue ($T_a$) - what you’d think of as “just temperature”, or ambient temperature. Does not account for radiation or humidity.

Thermodynamic Wet Bulb Temperature ($T_{pwb}$) wet bulb temperature in the shade and fanned or rotated. This is the wet bulb typically used for dew point calculations.

Natural Wet Bulb Temperature ($T_{nwb}$) wet bulb temperature with exposure to wind and sun. This is not a readily accessible measurement.

Globe Temperature ($T_g$) temperature taken from inside a copper globe painted black and exposed to the sun. Also not a readily accessible measurement.


$WBGT_{od} = 0.7*T_{nwb} + 0.2*T_g + 0.1*T_a$

$WBGT_{id} = 0.7*T_{nwb} + 0.3*T_g$

In [37]:
import math
import scipy
import xarray as xr
import numpy as np
import pandas as pd


In [13]:
def _calc_saturation_vapor_pressure(T_a): # Magnus-Tetens Approximation
    e_sat = 6.11 * math.exp((17.625 * T_a) / (T_a + 243.04)) # Saturation Vapor Pressure in hPa
    return e_sat

def _calc_relative_humidity_era5(T_a, T_d):
    e_sat = _calc_saturation_vapor_pressure(T_a) # saturation vapor pressure
    e = 6.11*math.exp(17.625 * T_d / (243.04 + T_d)) # vapor pressure from dew point temp
    rh = 100 * e / e_sat  # Clausius-Clapeyron equation
    return rh

def _calc_relative_humidity_CESM(T_a, Q, P):
    e_sat = _calc_saturation_vapor_pressure(T_a) # saturation vapor pressure
    w = Q / (1 - Q) # mixing ratio from specific humidity
    e = (w * P) / (0.622 + w) # vapor pressure from mixing ratio and pressure
    rh = 100 * e / e_sat  # Clausius-Clapeyron equation
    return rh

In [14]:
era5_file = "../../adaptor.mars.internal-1720716378.8156943-28180-13-3a40fe37-0128-4536-9777-7f2ecb6c7742.nc"
era5 = xr.open_dataset(era5_file)
era5

<xarray.Dataset> Size: 222MB
Dimensions:    (longitude: 237, latitude: 105, time: 744)
Coordinates:
  * longitude  (longitude) float32 948B -125.0 -124.8 -124.5 ... -66.25 -66.0
  * latitude   (latitude) float32 420B 50.0 49.75 49.5 49.25 ... 24.5 24.25 24.0
  * time       (time) datetime64[ns] 6kB 1995-07-01 ... 1995-07-31T23:00:00
Data variables:
    d2m        (time, latitude, longitude) float32 74MB ...
    t2m        (time, latitude, longitude) float32 74MB ...
    sp         (time, latitude, longitude) float32 74MB ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-07-11 16:46:23 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

## Liljergen (Argonne Model)

This is a method of approximating Globe Temperature ($T-g$).

This model:

- assumes that when there is no solar radiation, 
- assumes $T_{pwb}$ = $T_{nwb}$
- and tends to underestimate indoor $T_{nwb}$

In [21]:
def _calc_theta(lat, lon, datetime): # solar zenith angle
    phi = lat 

    timestamp = pd.Timestamp(era5_chicago.sel(datetime['values'])) # change to be more universal
    doy = timestamp.day_of_year # day of year
    time = timestamp.hour
    
    delta = 23.45  * math.sin(math.radians(360/365 * (284 + doy))) # solar declination

    B = 360/365 * (doy - 81)
    EOT = 9.87 * math.sin(2 * math.radians(B)) - 7.53 * math.cos(math.radians(B)) - 1.5 * math.sin(math.radians(B)) # equation of time
    time_correction = 4 * (lon - (-15 * round(lon / 15))) + EOT
    solar_time = time + time_correction / 60
    hour_angle = 15 * (12 - solar_time) # degrees

    cos_theta = (
        (math.sin(phi) * math.sin(delta))
        + (math.cos(phi) * math.cos(delta) * math.cos(math.radians(hour_angle)))
        )
    theta = math.acos(cos_theta)
    return theta

lat_chicago = 41.8781
lon_chicago = -87.6298

era5_chicago = era5.sel(latitude=lat_chicago, longitude=lon_chicago, method='nearest')
era5_chicago

<xarray.Dataset> Size: 15kB
Dimensions:    (time: 744)
Coordinates:
    longitude  float32 4B -87.75
    latitude   float32 4B 42.0
  * time       (time) datetime64[ns] 6kB 1995-07-01 ... 1995-07-31T23:00:00
Data variables:
    d2m        (time) float32 3kB ...
    t2m        (time) float32 3kB ...
    sp         (time) float32 3kB ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-07-11 16:46:23 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [ ]:
def _calc_earth_sun_distance(datetime):
    '''
    Assumes Earth's orbital velocity remains the same and that Earth's orbit's eccentricity is small.
    '''
    timestamp = pd.Timestamp(datetime['values'])
    doy = timestamp.day_of_year # day of year

    d = 1 - 0.0167 * math.cos(0.9856 * (doy - 4)) # Approximation form Kepler's first law
    return d


def _calc_solar_fraction(S, datetime, theta):
    f_dir = 0
    if theta <= 89.5:
        S_0 = 1367 # W/m^2
        d =  _calc_earth_sun_distance(datetime) # AUs
        S_max = S_0 * math.cos(theta) / d**2
        S_star = S / S_max
        f_dir = math.exp(3 - 1.34 * S_star - 1.65 / S_star)
    return f_dir

def _calc_horizontal_solar_irradiance():
    return S

In [ ]:

S = ' ' # total horizontal solar irradiance

def calc_liljergen_wet_bulb(T_a, # Ambient Temperature)
                            rh, # Relative Humidity
                            h, # Convective Heat Transfer Coefficient
                            lat # Latitude
                            lon, # Longitude
                            datetime, # Time of Day
                            ):
    epsilon_g = 0.95 # Globe emissivity
    alpha_sfc = 0.45 # Surface albedo
    sigma = scipy.constants.Stefan_Boltzmann # Stefan-Boltzmann Constant

    e_sat = _calc_saturation_vapor_pressure(T_a) # Saturation Vapor Pressure
    epsilon_a = 0.575 * (rh * e_sat) ** 0.143 # Atmosphere Emissivity

    theta = _calc_theta(lat, lon, datetime) # Solar Zenith Angle
    f_dir = _calc_solar_fraction(S, datetime, theta) # fraction due to diffuse solar irradiance


    # Convergence parameters
    tolerance = 1e-6
    max_iterations = 100

    # Iterative relaxation solve
    T_g = T_a
    for i in range(max_iterations):

        T_g_new = (
            0.5 * (1 + epsilon_a) * T_a ** 4
            - h / (epsilon_g * sigma) * (T_g - T_a)
            + S / (2 * epsilon_g * sigma) * (1 + f_dir * (1 / (2 * math.cos(theta)) - 1) + alpha_sfc)
        ) ** 0.25
        
        # Check for convergence
        if abs(T_g_new - T_g) < tolerance:
            T_g = T_g_new
            break
        
        T_g = T_g_new
    return T_g


def calc_liljergen_wbgt(T_a, T_nwb, *kwargs):
    T_g = calc_liljergen_wet_bulb(*kwargs)
    WBGT = 0.7*T_nwb + 0.2*T_g + 0.1*T_a
    return WBGT

## Bernard

This is a method of estimating indoor Wet Bulb Globe Temperature ($WBGT_{id}$) that does not consider sunny conditions.

In [ ]:
def calc_bernard_wbgt(t_a, t_pwb, v):
    if v < 0.3: #m/s
        return
    elif 0.3 <= v <= 3:
        wbgt = (0.67 * t_pwb) + (0.33 * t_a) - (0.48 * math.log10(v) * (t_a - t_pwb))
    else:
        wbgt = (0.7 * t_pwb) + (0.3 * t_a)
    return wbgt

## Australian Bureau of Meteorology (ABM)

Meethod of estimating Wet Bulb Global Temperature

- tends to overpredict WBGT
- assumes full sunlight and light breeze

In [ ]:
def calc_abm_wbgt(t_a, rh):
    e = (rh / 100)**17.27 * (t_a/(337.7 * t_a)) # water vapor pressure [hPa]
    wbgt = (0.567 * t_a) + (0.393 * e) + 3.94
    return wbgt

## Li
 
Use CESM to bias correct ERA5, focus on the heatwave in 1995 July that resulted in over 700 deaths
Will limit spacially to North America in 1995 to create a plot like Figure 1 for ONE ensemble member, then limit to Chicago and loop through all ensemble members and make some sort of line plot comparing them?

CESM - 15-member ensemble CESM simulations following the RCP4.5 compared to 8.5
WBGT* = .7*Tw + 0.3 Ta (natural wet bulb temp Tw)

Bias correctiont akes 40 years of ERA5 (1979-2018) compared to CESM historical (1979-2005) and RCP8.5 (2006-2018), this can be limited to USA though. paper mentioins Liljegren method

In [10]:
import geocat.datafiles as gdf
import xarray as xr
import numpy as np

In [8]:
era5_file = "../../adaptor.mars.internal-1720716378.8156943-28180-13-3a40fe37-0128-4536-9777-7f2ecb6c7742.nc"
era5 = xr.open_dataset(era5_file)
era5

<xarray.Dataset> Size: 222MB
Dimensions:    (longitude: 237, latitude: 105, time: 744)
Coordinates:
  * longitude  (longitude) float32 948B -125.0 -124.8 -124.5 ... -66.25 -66.0
  * latitude   (latitude) float32 420B 50.0 49.75 49.5 49.25 ... 24.5 24.25 24.0
  * time       (time) datetime64[ns] 6kB 1995-07-01 ... 1995-07-31T23:00:00
Data variables:
    d2m        (time, latitude, longitude) float32 74MB ...
    t2m        (time, latitude, longitude) float32 74MB ...
    sp         (time, latitude, longitude) float32 74MB ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-07-11 16:46:23 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [12]:
def calculate_tw_from_rh(t, rh):
    T_w =  t * np.arctan(0.151977 * (rh + 8.313659)**0.5) \
        + np.arctan(t + rh)  \
        - np.arctan(rh - 1.676331) \
        + 0.00391838 * (rh)**1.5 * np.arctan(0.023101 * rh) \
        -  4.686035
    return T_w

era5_rh = calc_relative_humidity_era5(era5.t2m, era5.d2m)
era5_tw = calculate_tw(era5.t2m, era5_rh)
era5_tw

IndentationError: unexpected indent (179706010.py, line 3)

--------------

In [ ]:
cesm_path = 'path_to_cesm.nc'

# Load the datasets
cesm = xr.open_dataset(cesm_path)

In [ ]:
# Define the coordinates for Chicago (maybe don't limit to Chicago yet?, )
lat_chicago = 41.8781
lon_chicago = -87.6298

era5_chicago = era5.sel(latitude=lat_chicago, longitude=lon_chicago)
cesm_chicago = cesm.sel(latitude=lat_chicago, longitude=lon_chicago)


In [ ]:
# Calculate approximation for wet bulb from ERA5

def calculate_tw(t, td, p):
    es = 6.112 * np.exp(17.67 * td / (td + 243.5))
    e = 6.112 * np.exp(17.67 * t / (t + 243.5))
    tw = t * np.arctan(0.151977 * np.sqrt(e + 8.313659)) + np.arctan(t + td) - np.arctan(td - 1.676331) + 0.00391838 * (e ** 1.5) * np.arctan(0.023101 * e) - 4.686035
    return tw

era5_tw_chicago = calculate_tw(era5_chicago.t2m, era5_chicago.d2m, era5_chicago.sp)


# NEED TO CALCULATE d2m from RELHUM first
cesm_tw_chicago = calculate_tw(cesm_chicago.t2m, cesm_chicago.d2m, cesm_chicago.sp)


In [ ]:
# Plug Wet Bulb Temp into standard WBGT equation

def calculate_wbgt(tw, ta):
    wbgt = 0.7 * tw + 0.3 * ta
    return wbgt

era5_wbgt_chicago = calculate_wbgt(era5_tw_chicago, era5_chicago.t2m)
cesm_wbgt_chicago = calculate_wbgt(cesm_tw_chicago, cesm_chicago.t2m)

In [ ]:
# Bias correction using the ∆-method

era5_seasonal_cycle = era5_wbgt_chicago.groupby('time.dayofyear').mean('time')
cesm_seasonal_cycle = cesm_wbgt_chicago.groupby('time.dayofyear').mean('time')

bias = cesm_seasonal_cycle - era5_seasonal_cycle

cesm_wbgt_corrected = cesm_wbgt_chicago.groupby('time.dayofyear') - bias

In [ ]:
# Plot